In [1]:
import numpy as np
import torch
from tqdm import tqdm

from torch import nn
from torch.optim import Adam
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import albumentations as A

from model.FoodDataset import FoodDataset
from model.architectures.UnetRestNet50 import UNetResNet152

torch.cuda.empty_cache()
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [2]:
image_paths = ["E:\Licenta_DOC\API_Segmentation\data\generated\img_dir"]
seg_paths = ["E:\Licenta_DOC\API_Segmentation\data\generated\\ann_dir"]

LOAD_MODEL = False
def get_images(transform = None,batch_size=32,shuffle=True,pin_memory=True):
    data = FoodDataset(image_paths, seg_paths,transform = transform)
    train_size = int(0.8 * data.__len__())
    test_size = data.__len__() - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])
    train_batch = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    test_batch = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)
    return train_batch,test_batch


In [3]:

t1 = A.Compose([
    A.Resize(256, 256),
    A.augmentations.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

train_batch,test_batch = get_images(transform =t1, batch_size=8)



In [4]:
# for img,mask in train_batch:
#     img1 = np.transpose(img[0,:,:,:],(1,2,0))
#     mask1 = np.array(mask[0,:,:])
#     img2 = np.transpose(img[1,:,:,:],(1,2,0))
#     mask2 = np.array(mask[1,:,:])
#     img3 = np.transpose(img[2,:,:,:],(1,2,0))
#     mask3 = np.array(mask[2,:,:])
#     fig , ax =  plt.subplots(3, 2, figsize=(18, 18))
#     ax[0][0].imshow(img1)
#     ax[0][1].imshow(mask1)
#     ax[1][0].imshow(img2)
#     ax[1][1].imshow(mask2)
#     ax[2][0].imshow(img3)
#     ax[2][1].imshow(mask3)
#     break

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [5]:
from model.checkpoints.checkpoints import load_checkpoint
from torchsummary import summary

num_classes = 104
model = UNetResNet152(num_classes).to(DEVICE)
MODEL_PRETRAINED_FILE = 'checkpoints/checkpoint-pretrain.pth.tar'
load_checkpoint(torch.load(MODEL_PRETRAINED_FILE), model)
print(summary(model,  (3,128, 128)))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11          [-1, 256, 32, 32]          16,384
      BatchNorm2d-12          [-1, 256, 32, 32]             512
           Conv2d-13          [-1, 256, 32, 32]          16,384
      BatchNorm2d-14          [-1, 256,

In [6]:
file = open("loss_data/unet_resnet.csv", "a")

In [7]:

from model.checkpoints.checkpoints import save_checkpoint
from model.losses.ce_dice_loss import CE_DICE_Loss

LEARNING_RATE = 1e-4
num_epochs = 15

loss_fn = CE_DICE_Loss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

for epoch in range(num_epochs):
    epoch_loss = 0.0

    loop = tqdm(enumerate(train_batch),total=len(train_batch))
    for batch_idx, (data, targets) in loop:
        data = data.to(DEVICE)
        targets = targets.to(DEVICE)
        targets = targets.type(torch.long)
        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        epoch_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    avg_loss = epoch_loss / loop.total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    file.write(f"{epoch+1},{avg_loss:.4f}\n")

file.close()
# save checkpoint
checkpoint =  {"state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()
           }
save_checkpoint(checkpoint, filename=MODEL_PRETRAINED_FILE)


  0%|          | 9/2492 [00:15<1:10:06,  1.69s/it, loss=1.11]


KeyboardInterrupt: 

In [ ]:

def check_accuracy(loader, model):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    iou = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            softmax = nn.Softmax(dim=1)
            preds = torch.argmax(softmax(model(x)),axis=1)
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
            intersection = torch.logical_and(preds, y).sum()
            union = torch.logical_or(preds, y).sum()
            iou += (intersection + 1e-8) / (union + 1e-8)

    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Dice score: {dice_score/len(loader)}")
    print(f"IoU: {iou/len(loader)}")
    model.train()



In [ ]:
print("Train")
check_accuracy(train_batch, model)

print("Test")
check_accuracy(test_batch, model)

In [ ]:

from skimage import color

for x,y in test_batch:
    x = x.to(DEVICE)
    fig , ax =  plt.subplots(3, 4, figsize=(72, 72))
    softmax = nn.Softmax(dim=1)
    preds = torch.argmax(softmax(model(x)),axis=1).to('cpu')
    img1 = np.transpose(np.array(x[0,:,:,:].to('cpu')),(1,2,0))
    preds1 = np.array(preds[0,:,:])
    mask1 = np.array(y[0,:,:])
    print("Pred1", np.unique(preds1))
    img2 = np.transpose(np.array(x[1,:,:,:].to('cpu')),(1,2,0))
    preds2 = np.array(preds[1,:,:])
    mask2 = np.array(y[1,:,:])
    # print("Pred2", np.unique(preds2))

    img3 = np.transpose(np.array(x[2,:,:,:].to('cpu')),(1,2,0))
    preds3 = np.array(preds[2,:,:])
    mask3 = np.array(y[2,:,:])
    # print("Pred3", np.unique(preds3))

    ax[0,0].set_title('Image')
    ax[0,1].set_title('Prediction')
    ax[0,2].set_title('Mask')
    ax[0,3].set_title('Overlay')
    ax[1,0].set_title('Image')
    ax[1,1].set_title('Prediction')
    ax[1,2].set_title('Mask')
    ax[1,3].set_title('Overlay')
    ax[2,0].set_title('Image')
    ax[2,1].set_title('Prediction')
    ax[2,2].set_title('Mask')
    ax[2,3].set_title('Overlay')
    ax[0][0].axis("off")
    ax[1][0].axis("off")
    ax[2][0].axis("off")
    ax[0][1].axis("off")
    ax[1][1].axis("off")
    ax[2][1].axis("off")
    ax[0][2].axis("off")
    ax[1][2].axis("off")
    ax[2][2].axis("off")
    ax[2][3].axis("off")
    ax[0][0].imshow(img1)
    ax[0][1].imshow(preds1)
    ax[0][2].imshow(mask1)
    ax[0][3].imshow(color.label2rgb(mask1, img1, saturation=1, alpha=0.5, bg_color=None))

    ax[1][0].imshow(img2)
    ax[1][1].imshow(preds2)
    ax[1][2].imshow(mask2)
    ax[2][0].imshow(img3)
    ax[2][1].imshow(preds3)
    ax[2][2].imshow(mask3)
    break

In [ ]:

from PIL import Image

x = np.array(Image.open('E:\Licenta_DOC\API_Segmentation\data\\train\img\\00000000.jpg'))

x = t1(image=x)['image']
x = x.to(DEVICE)

print(x.shape)
x = x.unsqueeze(0)
print(x.shape)
softmax = nn.Softmax(dim = 1)
preds = torch.argmax(softmax(model(x)), axis=1).to('cpu')
img1 = np.transpose(np.array(x[0, :, :, :].to('cpu')), (1, 2, 0))
preds1 = np.array(preds[0, :, :])
# 160 x 240
print(preds1)

fig , ax =  plt.subplots(2, 2, figsize=(18, 18))
ax[0,0].set_title('Image')
ax[0,1].set_title('Prediction')
ax[0][0].axis("off")
ax[1][0].axis("off")
ax[0][0].imshow(img1)
ax[0][1].imshow(preds1)
# ax[0][2].imshow(mask1)